---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [6]:
import re
import pandas as pd
import numpy as np
# list of unique states


def get_list_of_university_towns():
    df = pd.read_table('university_towns.txt', header = None)
    df.columns = ['Names']
    df['RegionName'] = df['Names'].apply(lambda x: x.split("(")[0].strip() if x.count('(') > 0 else np.NaN)
    df['State'] = df['Names'].apply(lambda x: x.split("[ed")[0].strip() if x.count('[ed') > 0 else np.NaN).fillna(method = 'ffill')
    for index, row in df.iterrows():
        if ':' in row['Names'] and row['Names'][-1]==":":
            row['RegionName'] = row['Names']
        if '[ed' not in row['Names'] and '(' not in row['Names']:
            row['RegionName'] = row['Names']
    df = df[['State','RegionName']]
    df.reset_index(inplace = True)
    del df['index']
    return df.dropna()

In [8]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    import pandas as pd

    df = pd.read_excel('gdplev.xls', skiprows = 5, parse_cols = [4,6], names = [['YEAR_QUARTER','GDP_IN_BIL']])
    df = df.dropna()
    df = df[df['YEAR_QUARTER'].str[:4]>='2000']
    seq_str='0'
    for i in range(1,len(df)):
        if df.iloc[i]['GDP_IN_BIL']>df.iloc[i-1]['GDP_IN_BIL']:
            seq_str += '1'
        if df.iloc[i]['GDP_IN_BIL']<df.iloc[i-1]['GDP_IN_BIL']:
            seq_str += '0'
    return df.iloc[seq_str.rindex('1',0,seq_str.index('0011'))+1]['YEAR_QUARTER']
get_recession_start()

'2008q3'

In [20]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import pandas as pd

    df = pd.read_excel('gdplev.xls', skiprows = 5, parse_cols = [4,6], names = [['YEAR_QUARTER','GDP_IN_BIL']])
    df = df.dropna()
    df = df[df['YEAR_QUARTER'].str[:4]>='2000']

    for i in range(1,len(df)):
        if i+3 > len(df)-1:
            continue
        if df.iloc[i-1]['GDP_IN_BIL'] > df.iloc[i]['GDP_IN_BIL'] and df.iloc[i]['GDP_IN_BIL'] > df.iloc[i+1]['GDP_IN_BIL'] and df.iloc[i+1]['GDP_IN_BIL'] < df.iloc[i+2]['GDP_IN_BIL'] and df.iloc[i+2]['GDP_IN_BIL'] < df.iloc[i+3]['GDP_IN_BIL']:
            return df.iloc[i+3]['YEAR_QUARTER']
get_recession_end()

'2009q4'

In [1]:
def get_recession_start_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import pandas as pd

    df = pd.read_excel('gdplev.xls', skiprows = 5, parse_cols = [4,6], names = [['YEAR_QUARTER','GDP_IN_BIL']])
    df = df.dropna()
    df = df[df['YEAR_QUARTER'].str[:4]>='2000']
    list1 = []
    
    seq_str='0'
    for i in range(1,len(df)):
        if df.iloc[i]['GDP_IN_BIL']>df.iloc[i-1]['GDP_IN_BIL']:
            seq_str += '1'
        if df.iloc[i]['GDP_IN_BIL']<df.iloc[i-1]['GDP_IN_BIL']:
            seq_str += '0'
    list1.append(df.iloc[seq_str.rindex('1',0,seq_str.index('0011'))+1]['YEAR_QUARTER'])
    
    
    for i in range(1,len(df)):
        if i+3 > len(df)-1:
            continue
        if df.iloc[i-1]['GDP_IN_BIL'] > df.iloc[i]['GDP_IN_BIL'] and df.iloc[i]['GDP_IN_BIL'] > df.iloc[i+1]['GDP_IN_BIL'] and df.iloc[i+1]['GDP_IN_BIL'] < df.iloc[i+2]['GDP_IN_BIL'] and df.iloc[i+2]['GDP_IN_BIL'] < df.iloc[i+3]['GDP_IN_BIL']:
            list1.append(df.iloc[i+3]['YEAR_QUARTER'])
            return list1

def get_recession_bottom():
    import pandas as pd
    list1 = get_recession_start_end()
    df = pd.read_excel('gdplev.xls', skiprows = 5, parse_cols = [4,6], names = [['YEAR_QUARTER','GDP_IN_BIL']])
    df = df.dropna()
    df = df[(df['YEAR_QUARTER']>=list1[0]) & (df['YEAR_QUARTER']<=list1[1])]
    min_val = min(df['GDP_IN_BIL'])
    df.where(df['GDP_IN_BIL'] == min_val, inplace = True)
    return list(df.dropna()['YEAR_QUARTER'])[0]
get_recession_bottom()

'2009q2'

In [ ]:

def convert_housing_data_to_quarters():
    import pandas as pd
    import numpy as np
    raw_df = pd.read_csv('City_Zhvi_AllHomes.csv').fillna(0)
    df = raw_df.iloc[:,np.r_[1,2,51:251]].copy()
    df['2016-09'] = 0
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 
              'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    df['states'] = df['State'].map(states)
    df.set_index(['states','RegionName'], inplace = True)
    df = df.iloc[:,1:]
    tdf = df.rename(columns = pd.to_datetime)
    mdf = tdf.resample('3M', axis = 1, closed = 'left').mean().rename(columns = lambda x: '{:}q{:}'.format(x.year,int(x.month/3)))
    return mdf

In [3]:


import re
import pandas as pd
import numpy as np
from scipy import stats

def get_list_of_university_towns():
    df = pd.read_table('university_towns.txt', header = None)
    df.columns = ['Names']
    df['RegionName'] = df['Names'].apply(lambda x: x.split("(")[0].strip() if x.count('(') > 0 else np.NaN)
    df['State'] = df['Names'].apply(lambda x: x.split("[ed")[0].strip() if x.count('[ed') > 0 else np.NaN).fillna(method = 'ffill')
    for index, row in df.iterrows():
        if ':' in row['Names'] and row['Names'][-1]==":":
            row['RegionName'] = row['Names']
        if '[ed' not in row['Names'] and '(' not in row['Names']:
            row['RegionName'] = row['Names']
    df = df[['State','RegionName']]
    df.reset_index(inplace = True)
    del df['index']
    return df.dropna()

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    import pandas as pd

    df = pd.read_excel('gdplev.xls', skiprows = 5, parse_cols = [4,6], names = [['YEAR_QUARTER','GDP_IN_BIL']])
    df = df.dropna()
    df = df[df['YEAR_QUARTER'].str[:4]>='2000']
    seq_str='0'
    for i in range(1,len(df)):
        if df.iloc[i]['GDP_IN_BIL']>df.iloc[i-1]['GDP_IN_BIL']:
            seq_str += '1'
        if df.iloc[i]['GDP_IN_BIL']<df.iloc[i-1]['GDP_IN_BIL']:
            seq_str += '0'
    return df.iloc[seq_str.rindex('1',0,seq_str.index('0011'))+1]['YEAR_QUARTER']



def get_recession_start_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import pandas as pd

    df = pd.read_excel('gdplev.xls', skiprows = 5, parse_cols = [4,6], names = [['YEAR_QUARTER','GDP_IN_BIL']])
    df = df.dropna()
    df = df[df['YEAR_QUARTER'].str[:4]>='2000']
    list1 = []
    
    seq_str='0'
    for i in range(1,len(df)):
        if df.iloc[i]['GDP_IN_BIL']>df.iloc[i-1]['GDP_IN_BIL']:
            seq_str += '1'
        if df.iloc[i]['GDP_IN_BIL']<df.iloc[i-1]['GDP_IN_BIL']:
            seq_str += '0'
    list1.append(df.iloc[seq_str.rindex('1',0,seq_str.index('0011'))+1]['YEAR_QUARTER'])
    
    
    for i in range(1,len(df)):
        if i+3 > len(df)-1:
            continue
        if df.iloc[i-1]['GDP_IN_BIL'] > df.iloc[i]['GDP_IN_BIL'] and df.iloc[i]['GDP_IN_BIL'] > df.iloc[i+1]['GDP_IN_BIL'] and df.iloc[i+1]['GDP_IN_BIL'] < df.iloc[i+2]['GDP_IN_BIL'] and df.iloc[i+2]['GDP_IN_BIL'] < df.iloc[i+3]['GDP_IN_BIL']:
            list1.append(df.iloc[i+3]['YEAR_QUARTER'])
            return list1

def get_recession_bottom():
    import pandas as pd
    list1 = get_recession_start_end()
    df = pd.read_excel('gdplev.xls', skiprows = 5, parse_cols = [4,6], names = [['YEAR_QUARTER','GDP_IN_BIL']])
    df = df.dropna()
    df = df[(df['YEAR_QUARTER']>=list1[0]) & (df['YEAR_QUARTER']<=list1[1])]
    min_val = min(df['GDP_IN_BIL'])
    df.where(df['GDP_IN_BIL'] == min_val, inplace = True)
    return list(df.dropna()['YEAR_QUARTER'])[0]

def convert_housing_data_to_quarters():
    import pandas as pd
    import numpy as np
    raw_df = pd.read_csv('City_Zhvi_AllHomes.csv').fillna(0)
    df = raw_df.iloc[:,np.r_[1,2,51:251]].copy()
    df['2016-09'] = 0
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 
              'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    df['states'] = df['State'].map(states)
    df.set_index(['states','RegionName'], inplace = True)
    df = df.iloc[:,1:]
    tdf = df.rename(columns = pd.to_datetime)
    mdf = tdf.resample('3M', axis = 1, closed = 'left').mean().rename(columns = lambda x: '{:}q{:}'.format(x.year,int(x.month/3)))
    return mdf

def run_ttest():
    
    hdf = convert_housing_data_to_quarters()
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()
    ul = get_list_of_university_towns()

    recession_start = hdf.columns.get_loc(rec_start)
    recession_bottom = hdf.columns.get_loc(rec_bottom)


    hdf['PriceRatio'] = hdf.iloc[:,recession_start]/hdf.iloc[:,recession_bottom]
    tuple_list = [tuple(x) for x in ul.values]
    raw_university_towns = hdf.loc[tuple_list].fillna(0)
    #print(raw_university_towns)
    university_towns = raw_university_towns.dropna(subset=['PriceRatio'])
    raw_non_university_towns = hdf.loc[~hdf.index.isin(tuple_list)].fillna(0)
    non_university_towns = raw_non_university_towns.dropna(subset=['PriceRatio'])
    university_towns_mean = raw_university_towns['PriceRatio'].mean()
    non_university_towns_mean = raw_non_university_towns['PriceRatio'].mean()


    pvalue = list(stats.ttest_ind(raw_university_towns['PriceRatio'], raw_non_university_towns['PriceRatio']))[1]
    different = pvalue <0.01
    if non_university_towns_mean < university_towns_mean:
        better = 'non-university town'
    else: better = 'university town'
    return (different,pvalue,better)
run_ttest()
#(False, 0.046040975395856894, 'university town')

(True, 5.5645761040968171e-218, 'university town')